# Image training

In [1]:
import torch

In [2]:
from torch.serialization import add_safe_globals
from torch.utils.data import TensorDataset

add_safe_globals([TensorDataset])

train_dataset = torch.load('../data/processed/train_dataset.pt', weights_only=False)
test_dataset = torch.load('../data/processed/test_dataset.pt', weights_only=False)


In [3]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [4]:
import sys
sys.path.append('../src')

from trainer  import BasicCNN, get_resnet18_transfer_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate both models
model_basic = BasicCNN().to(device)
model_resnet18 = get_resnet18_transfer_model().to(device)


In [11]:
from trainer import train_and_evaluate
# Train BasicCNN
model_basic, acc_basic = train_and_evaluate(model_basic, train_loader, test_loader, num_epochs=15, device=device)

# Train ResNet18
model_resnet18, acc_resnet = train_and_evaluate(model_resnet18, train_loader, test_loader, num_epochs=15, device=device)

# Compare results
print(f"✅ BasicCNN Accuracy: {acc_basic:.4f}")
print(f"✅ ResNet18 Accuracy: {acc_resnet:.4f}")


Epoch 1/15, Training Loss: 0.8817
Epoch 2/15, Training Loss: 0.6763
Epoch 3/15, Training Loss: 0.6560
Epoch 4/15, Training Loss: 0.6426
Epoch 5/15, Training Loss: 0.6399
Epoch 6/15, Training Loss: 0.6365
Epoch 7/15, Training Loss: 0.6339
Epoch 8/15, Training Loss: 0.6335
Epoch 9/15, Training Loss: 0.6332
Epoch 10/15, Training Loss: 0.6330
Epoch 11/15, Training Loss: 0.6330
Epoch 12/15, Training Loss: 0.6329
Epoch 13/15, Training Loss: 0.6329
Epoch 14/15, Training Loss: 0.6329
Epoch 15/15, Training Loss: 0.6329
Final Test Accuracy: 0.6269
Epoch 1/15, Training Loss: 0.6894
Epoch 2/15, Training Loss: 0.3660
Epoch 3/15, Training Loss: 0.1312
Epoch 4/15, Training Loss: 0.0500
Epoch 5/15, Training Loss: 0.0591
Epoch 6/15, Training Loss: 0.0208
Epoch 7/15, Training Loss: 0.0139
Epoch 8/15, Training Loss: 0.0103
Epoch 9/15, Training Loss: 0.0158
Epoch 10/15, Training Loss: 0.0075
Epoch 11/15, Training Loss: 0.0104
Epoch 12/15, Training Loss: 0.0081
Epoch 13/15, Training Loss: 0.0081
Epoch 14/1

In [5]:
import sys
sys.path.append('../src')
from trainer import get_vit_model
model_vit = get_vit_model(num_classes=2)

In [7]:

from trainer import train_and_evaluate
model_vit, acc_vit = train_and_evaluate(
    model_vit,
    train_loader,
    test_loader,
    num_epochs=20,
    lr=1e-4,
    device=device
)

print(f"✅ Vision Transformer Accuracy: {acc_vit:.4f}")

Epoch 1/20, Training Loss: 0.3970
Epoch 2/20, Training Loss: 0.3904
Epoch 3/20, Training Loss: 0.3852
Epoch 4/20, Training Loss: 0.3817
Epoch 5/20, Training Loss: 0.3812
Epoch 6/20, Training Loss: 0.3807
Epoch 7/20, Training Loss: 0.3804
Epoch 8/20, Training Loss: 0.3804
Epoch 9/20, Training Loss: 0.3803
Epoch 10/20, Training Loss: 0.3803
Epoch 11/20, Training Loss: 0.3803
Epoch 12/20, Training Loss: 0.3803
Epoch 13/20, Training Loss: 0.3803
Epoch 14/20, Training Loss: 0.3803
Epoch 15/20, Training Loss: 0.3803
Epoch 16/20, Training Loss: 0.3803
Epoch 17/20, Training Loss: 0.3803
Epoch 18/20, Training Loss: 0.3803
Epoch 19/20, Training Loss: 0.3803
Epoch 20/20, Training Loss: 0.3803
Final Test Accuracy: 0.7612
✅ Vision Transformer Accuracy: 0.7612


In [8]:
# Save best model
torch.save(model_resnet18.state_dict(), "../models/resnet18_best.pth")


# Patch training

In [1]:
import torch
from torch.serialization import add_safe_globals
from torch.utils.data import TensorDataset

add_safe_globals([TensorDataset])

train_dataset = torch.load('../data/processed/train_patch_dataset.pt', weights_only=False)
test_dataset = torch.load('../data/processed/test_patch_dataset.pt', weights_only=False)

from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [2]:
import sys
sys.path.append('../src')
from trainer import patch_train_and_evaluate, get_model


In [ ]:
trained_models = {}
f1_scores = {}

for model_name in ['resnet', 'vgg', 'efficientnet']:
    print(f"\n📌 Training {model_name.upper()}...")
    model = get_model(model_name)
    trained_model, f1 = patch_train_and_evaluate(model, train_loader, test_loader)
    trained_models[model_name] = trained_model
    f1_scores[model_name] = f1
    print(f"🎯 {model_name.upper()} F1 Score: {f1:.4f}")

# Identify best model
best_model_name = max(f1_scores, key=f1_scores.get)
print(f"\n🏆 Best model: {best_model_name.upper()} with F1 Score {f1_scores[best_model_name]:.4f}")

# Save it
torch.save(trained_models[best_model_name].state_dict(), f'../models/{best_model_name}_best_patch.pth')



📌 Training RESNET...
Epoch 1/10, Training Loss: 0.3801
Epoch 2/10, Training Loss: 0.2734
Epoch 3/10, Training Loss: 0.2450
Epoch 4/10, Training Loss: 0.2652
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 5/10, Training Loss: 0.2240
Epoch 6/10, Training Loss: 0.2225
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 7/10, Training Loss: 0.1832
Epoch 8/10, Training Loss: 0.1708
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 9/10, Training Loss: 0.1555
Epoch 10/10, Training Loss: 0.1485
⚠️  Minor improvment improvement for 1 epoch(s).
✅ Final F1 Score: 0.8969
🎯 RESNET F1 Score: 0.8969

📌 Training VGG...


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Krish/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:43<00:00, 12.7MB/s] 


Epoch 1/10, Training Loss: 1.0415
Epoch 2/10, Training Loss: 0.5122
Epoch 3/10, Training Loss: 0.3918
Epoch 4/10, Training Loss: 0.3162
Epoch 5/10, Training Loss: 0.2979
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 6/10, Training Loss: 0.4732
⚠️  Minor improvment improvement for 2 epoch(s).
Epoch 7/10, Training Loss: 0.6593
⚠️  Minor improvment improvement for 3 epoch(s).
⏹️  Early stopping triggered.
✅ Final F1 Score: 0.4831
🎯 VGG F1 Score: 0.4831

📌 Training EFFICIENTNET...


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Krish/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:03<00:00, 5.38MB/s]


Epoch 1/10, Training Loss: 0.2215
Epoch 2/10, Training Loss: 0.1194
Epoch 3/10, Training Loss: 0.0845
Epoch 4/10, Training Loss: 0.0645
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 5/10, Training Loss: 0.0629
Epoch 6/10, Training Loss: 0.0439
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 7/10, Training Loss: 0.0415
Epoch 8/10, Training Loss: 0.0222
⚠️  Minor improvment improvement for 1 epoch(s).
Epoch 9/10, Training Loss: 0.0186
Epoch 10/10, Training Loss: 0.0348
⚠️  Minor improvment improvement for 1 epoch(s).
✅ Final F1 Score: 0.9062
🎯 EFFICIENTNET F1 Score: 0.9062


In [5]:
torch.save(model.state_dict(), '../models/efficientnet_best_patch.pth')
